# Image Classification

## Data set: 1. MNIST; 2. Cifar-10; 3. classify-leaves

In [1]:
import torch
import torch.nn as nn
from torch.functional import F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'svg'

In [2]:
use_cuda = True
device = torch.device('cuda:0' if use_cuda else 'cpu')

In [3]:
mytransforms = transforms.Compose([transforms.ToTensor()])
MNIST_data_train = torchvision.datasets.MNIST(root = 'data', train = True, download = True, transform = mytransforms)
MNIST_dataloader_train = torch.utils.data.DataLoader(MNIST_data_train, batch_size = 256, shuffle = True)
MNIST_data_test = torchvision.datasets.MNIST(root = 'data', train = False, download = True, transform = mytransforms)
MNIST_dataloader_test = torch.utils.data.DataLoader(MNIST_data_test, batch_size = 256, shuffle = False)

In [4]:
x,y = iter(MNIST_dataloader_train).next()
x = x.cuda(device)
y = y.cuda(device)

In [5]:
class BasicBlock(nn.Module):
    def __init__(self, input_channels, num_channels, stride = 1, downsample = None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, num_channels, 3, stride = 1, padding = 1)
        self.conv2 = nn.Conv2d(num_channels, num_channels, 3, stride = stride, padding = 1)
        self.conv3 = nn.Conv2d(input_channels, num_channels, 1, stride = stride)
        
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)
        
        self.relu = nn.ReLU(inplace = True)
        
    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        out = F.relu(out)
        
        x = self.conv3(x)
        
        Y = F.relu(out + x)
        return Y

In [6]:
def Resnet_block(input_channels, num_channels, num_layers):
    blk = []
    for i in range(num_layers):
        blk.append(BasicBlock(input_channels, num_channels, stride = 2))
        input_channels = num_channels
    return blk

In [7]:
class Resnet_MNIST(nn.Module):
    def __init__(self, input_channels, num_channels):
        super(Resnet_MNIST, self).__init__()
        self.b1 = nn.Sequential(
            nn.Conv2d(input_channels, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace = True),
        )
        self.b2 = nn.Sequential(*Resnet_block(64,64,2))
        self.b3 = nn.Sequential(*Resnet_block(64,128,2))
        self.b4 = nn.Sequential(*Resnet_block(128,256,2))
        self.b5 = nn.Sequential(*Resnet_block(256,512,2))
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.flatten = nn.Flatten()
        self.linear = nn.Linear(512, num_channels)
        self.softmax = nn.Softmax(dim = 1)
        
        
    def forward(self, x):
        x = self.b1(x)
        x = self.b2(x)
        x = self.b3(x)
        x = self.b4(x)
        x = self.b5(x)
        x = self.avgpool(x)
        x = self.flatten(x)
        x = self.linear(x)
        out = self.softmax(x)
        return out

In [8]:
myResnet_MNIST = Resnet_MNIST(1,10)
myResnet_MNIST.to(device)
myResnet_MNIST.training

True

In [9]:
x,y = iter(MNIST_dataloader_train).next()
x = x.cuda(device)
y = y.cuda(device)
y_ = myResnet_MNIST(x)

In [10]:
torch.argmax(y_, 1)

tensor([1, 8, 7, 8, 1, 0, 1, 8, 7, 0, 8, 8, 1, 1, 8, 1, 8, 1, 1, 1, 1, 8, 8, 8,
        1, 1, 3, 8, 7, 8, 8, 8, 8, 1, 2, 8, 8, 1, 8, 8, 7, 5, 8, 8, 1, 9, 1, 8,
        8, 8, 1, 1, 1, 8, 8, 7, 3, 2, 1, 8, 1, 1, 2, 1, 8, 8, 7, 8, 8, 1, 8, 7,
        1, 0, 8, 1, 1, 8, 1, 2, 7, 8, 1, 8, 1, 8, 8, 3, 8, 8, 8, 1, 1, 8, 7, 3,
        8, 8, 1, 8, 8, 8, 1, 5, 8, 8, 3, 8, 1, 8, 8, 8, 8, 8, 8, 2, 1, 8, 8, 8,
        1, 1, 1, 1, 1, 1, 1, 8, 8, 8, 8, 7, 8, 8, 0, 3, 1, 7, 3, 2, 8, 1, 3, 1,
        1, 8, 8, 8, 3, 1, 8, 2, 1, 1, 0, 8, 8, 2, 8, 1, 8, 5, 2, 1, 8, 1, 8, 8,
        1, 8, 1, 8, 2, 1, 2, 1, 8, 8, 8, 7, 8, 8, 1, 7, 1, 7, 3, 1, 1, 8, 7, 1,
        1, 1, 8, 1, 7, 3, 8, 2, 8, 1, 1, 8, 1, 1, 8, 1, 1, 8, 3, 1, 1, 1, 8, 1,
        5, 2, 1, 8, 8, 1, 2, 5, 8, 3, 1, 1, 8, 2, 2, 5, 8, 8, 8, 1, 8, 8, 8, 3,
        8, 8, 1, 1, 1, 8, 1, 1, 8, 2, 1, 7, 1, 8, 2, 1], device='cuda:0')

In [11]:
optimizer = optim.Adam(myResnet_MNIST.parameters(), lr = 0.001)
loss_func = nn.CrossEntropyLoss()

In [12]:
myResnet_MNIST.train()
for epoch in range(0):
    for step,(x,y) in enumerate(MNIST_dataloader_train):
        x = x.cuda(device)
        y = y.cuda(device)
        y_ = myResnet_MNIST(x)
        
        loss = loss_func(y_,y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        acc = torch.sum(torch.argmax(y_, 1) == y)/y.shape[0]
        print('\r epoch:{epoch:5}--step:{step:7}--loss:{loss:.10}--acc:{acc:.5}'.format(epoch=epoch, step=step, loss=loss,acc = acc),end = '')

In [13]:
# torch.save(myResnet_MNIST, 'model/Resnet_MNIST.pt')
myResnet_MNIST_test = torch.load('model/Resnet_MNIST.pt')

In [14]:
myResnet_MNIST_test.eval()
den = 0
num = 0
for step,(x,y) in enumerate(MNIST_dataloader_test):
    x = x.cuda(device)
    y = y.cuda(device)
    y_ = myResnet_MNIST_test(x)
    num += torch.sum(torch.argmax(y_, 1) == y)
    den += y.shape[0]
    print('\r',step,end='')
acc = num/den
acc

 39

tensor(0.9560, device='cuda:0')

In [15]:
mytransforms = transforms.Compose([transforms.ToTensor()])
Cifar10_data_train = torchvision.datasets.CIFAR10(root = 'data', train = True, download = True, transform = mytransforms)
Cifar10_dataloader_train = torch.utils.data.DataLoader(Cifar10_data_train, batch_size = 256, shuffle = True)
Cifar10_data_test = torchvision.datasets.CIFAR10(root = 'data', train = False, download = True, transform = mytransforms)
Cifar10_dataloader_test = torch.utils.data.DataLoader(Cifar10_data_test, batch_size = 256, shuffle = False)

Cifar100_data_train = torchvision.datasets.CIFAR100(root = 'data', train = True, download = True, transform = mytransforms)
Cifar100_dataloader_train = torch.utils.data.DataLoader(Cifar100_data_train, batch_size = 256, shuffle = True)
Cifar100_data_test = torchvision.datasets.CIFAR100(root = 'data', train = False, download = True, transform = mytransforms)
Cifar100_dataloader_test = torch.utils.data.DataLoader(Cifar100_data_test, batch_size = 256, shuffle = False)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
